In [1]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))
        except:
            try:
                t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","Ours Individual","2", "3"]
            t.loc[k+1,"Train"]=calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[k])
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt"):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
        t.loc["Ann(tr)",:] = aux
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(4), t_std.round(4)

!ls

I_fix  I_fix0  LabelMe	LabelMe_v2  Plot_Results.ipynb	simple_T


In [11]:
!ls I_fix/LabelMe
folder = "I_fix/LabelMe_v2/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

LabelMe_hardMV_test.csv
LabelMe_hardMV_test_std.csv
LabelMe_hardMV_train.csv
LabelMe_hardMV_train_std.csv
LabelMe_OursIndividual2_testAux.csv
LabelMe_OursIndividual2_testAux_std.csv
LabelMe_OursIndividual2_test.csv
LabelMe_OursIndividual2_test_std.csv
LabelMe_OursIndividual2_trainAnn.csv
LabelMe_OursIndividual2_trainAnn_std.csv
LabelMe_OursIndividual2_train.csv
LabelMe_OursIndividual2_train_std.csv
LabelMe_OursIndividual3_testAux.csv
LabelMe_OursIndividual3_testAux_std.csv
LabelMe_OursIndividual3_test.csv
LabelMe_OursIndividual3_test_std.csv
LabelMe_OursIndividual3_trainAnn.csv
LabelMe_OursIndividual3_trainAnn_std.csv
LabelMe_OursIndividual3_train.csv
LabelMe_OursIndividual3_train_std.csv
LabelMe_softMV_test.csv
LabelMe_softMV_test_std.csv
LabelMe_softMV_train.csv
LabelMe_softMV_train_std.csv
LabelMe_UpperBound_test.csv
LabelMe_UpperBound_train.csv
results_ind.txt


['OursIndividual2', 'OursIndividual3', 'softMV', 'hardMV']

In [12]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [13]:
t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)   0.769       -   0.769       -
1  OursIndividual2  0.8161  0.8325  0.0078  0.0099
2  OursIndividual3  0.7729  0.7762  0.0193    0.02
3           softMV  0.8359  0.8184  0.0083   0.011
4           hardMV  0.8051  0.8001  0.0083  0.0096

In [14]:
from IPython.display import display
display(t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_R.merge(t_norm_R_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividual2  0.3587      -  0.0041      -
2  OursIndividual3  0.2992      -  0.0159      -
3           softMV  0.4632      -       0      -
4           hardMV  0.4924      -       0      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividual2   0.232      -  0.0015      -
2  OursIndividual3  0.2175      -  0.0062      -
3           softMV  0.2636      -       0      -
4           hardMV  0.2739      -       0      -

In [26]:
display(t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_G.merge(t_norm_G_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1           softMV  0.0242      -       0      -
2  OursIndividual2  0.0512      -  0.0032      -
3           hardMV  0.0347      -       0      -
4  OursIndividual3  0.0517      -   0.004      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1           softMV  0.0566      -       0      -
2  OursIndividual2  0.0545      -  0.0026      -
3           hardMV  0.0736      -       0      -
4  OursIndividual3  0.0674      -  0.0036      -

## T change

In [2]:
folder = "./I_fix0/simple_T/"

In [3]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['hardMV', 'OursIndividualK', 'softMV', 'OursIndividualT']

In [15]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results_ind.txt')
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
Ann(tr)          0.7452  0.6996  0.7118  0.7196  0.7208   0.7061  0.0000   
OursIndividualT  0.9454  0.9441  0.9003  0.9192  0.8915   0.8640  0.0028   
OursIndividualK  0.9443  0.9441  0.9396  0.9206  0.8727   0.7871  0.0027   
hardMV           0.8457  0.7510  0.7959  0.8236  0.8164   0.7896  0.0420   
softMV           0.6983  0.6584  0.6738  0.6823  0.6810   0.6712  0.0200   

                 1500_y  3500_y  6000_y  10000_y  
Ann(tr)          0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0412  0.0048  0.0129   0.0248  
OursIndividualK  0.0061  0.0155  0.0448   0.1172  
hardMV           0.0517  0.0550  0.0407   0.0551  
softMV           0.0484  0.0402  0.0483   0.0385

In [7]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean")
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
hardMV           0.0797  0.0834  0.0933  0.0877  0.0850   0.0867  0.0000   
OursIndividualK  0.0160  0.0200  0.0517  0.0919  0.1442   0.1984  0.0032   
softMV           0.0803  0.0850  0.0903  0.0861  0.0859   0.0876  0.0000   
OursIndividualT  0.0174  0.0208  0.0532  0.1145  0.1345   0.2045  0.0017   

                 1500_y  3500_y  6000_y  10000_y  
hardMV           0.0000  0.0000  0.0000   0.0000  
OursIndividualK  0.0082  0.0043  0.0095   0.0162  
softMV           0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0044  0.0014  0.0137   0.0139

In [8]:
t, t_std = create_df(methods_list,to_check,"(G) JS")
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
hardMV           0.0151  0.0149  0.0203  0.0184  0.0178   0.0163  0.0000   
OursIndividualK  0.0058  0.0029  0.0052  0.0081  0.0153   0.0402  0.0013   
softMV           0.0135  0.0135  0.0149  0.0146  0.0149   0.0142  0.0000   
OursIndividualT  0.0039  0.0019  0.0106  0.0042  0.0053   0.0164  0.0011   

                 1500_y  3500_y  6000_y  10000_y  
hardMV           0.0000  0.0000  0.0000   0.0000  
OursIndividualK  0.0047  0.0042  0.0025   0.0109  
softMV           0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0037  0.0002  0.0037   0.0017

In [ ]:
print(tabulate(t,headers='keys',tablefmt='latex'))

In [40]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100     500     1500    3500    6000    10000
Ann(tr)          0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
DS               0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
OursIndividual3  0.2886  0.3125  0.3407  0.4686  0.5989  0.7124
hardMV           0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
OursIndividual2  0.2879  0.2891  0.2336  0.3825  0.5453  0.6709
softMV           0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
OursGlobal       0.2297  0.2429  0.2102  0.2404  0.2049  0.1949
Raykar           0.0000  0.0000  0.0000  0.0000  0.0000  0.0000